##**Required Libraries to Install**

In [3]:
!pip install pymupdf huggingface_hub

##**Provide Huggingface API Key Here**

In [5]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Paste token here"


##**PDF DATA EXTRACTION USING LLM**

In [6]:
# Install dependencies


import fitz  # PyMuPDF
import base64
import os
from huggingface_hub import InferenceClient
from google.colab import files

# Step 1: Upload PDF
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Step 2: Convert PDF pages to base64 images
def pdf_to_base64_pages(pdf_path, dpi=150, img_format="png"):
    doc = fitz.open(pdf_path)
    base64_pages = []

    for page in doc:
        pix = page.get_pixmap(dpi=dpi)
        img_bytes = pix.tobytes(img_format)
        img_b64 = base64.b64encode(img_bytes).decode("utf-8")
        base64_pages.append(f"data:image/{img_format};base64,{img_b64}")

    return base64_pages

pages_b64 = pdf_to_base64_pages(pdf_path)

# Step 3: Initialize Hugging Face InferenceClient
client = InferenceClient(
    provider="novita",
    api_key=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)


prompt="""You are an intelligent image analysis assistant who can extract information from the images also those images that are embedded within the image. Your task is to extract information from images as accurately and completely as possible. You should detect text, tables, and embedded images, and provide concise descriptions for regular images. Always follow the output format strictly, even if some sections are empty.



Analyze the provided image and extract information according to the following rules:



###1.Textual Information:

(In this section you will provide the textual information as it is in the picture)

Extract all visible text exactly as it appears in the image.

Preserve line breaks, punctuation, and order. if it is double columned provide in single column.



###2.Tables:


(In this section you will provide all tables also those tables that are in embedded images)

Extract all tables in a structured format.

Include tables that are embedded inside images or diagrams.

If tables have titles, include them.(If it doesn't have any title provide one according to table information)




###3.Textual Information in Embedded Images:

(In this section you will provide all the textual information that are in embedded images with numbering serially.)

Detect any text present within images inside the main image (e.g., charts, signs, screenshots).

Extract the text exactly as it appears.





###4.Images Short Description:

(In this section you will provide the short descriptions of the regular images.)

For any regular embedded images, provide a very short description (e.g., “Image 1: a bar chart(short description),” “Image 2: a landscape photo(short description)”).




Important Rules:

Always output all four sections, even if some are blank.Just provide "None" there.

Do not merge sections.

Do not add extra explanations or commentary.

Preserve the exact structure below:

"""



# Step 4: Send pages one by one to model
for i, page_b64 in enumerate(pages_b64, start=1):
    completion = client.chat.completions.create(
        model="Qwen/Qwen3-VL-235B-A22B-Instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": page_b64}
                    }
                ]
            }
        ],
    )
    print(f"\n--- Page {i} ---\n")
    print(completion.choices[0].message.content)


Saving Test PDF.pdf to Test PDF.pdf


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



--- Page 1 ---

###1.Textual Information:
Machine Learning: An Overview

Introduction
Machine Learning (ML) is a subset of Artificial Intelligence (AI) that enables systems to learn and improve from experience without being explicitly programmed. Unlike traditional programming, where rules are hard-coded, ML algorithms identify patterns in data and make predictions or decisions based on those patterns.

Machine Learning has transformed various industries, including healthcare, finance, retail, and autonomous vehicles. The rapid increase in computational power, availability of big data, and advanced algorithms have fueled its growth.

Types of Machine Learning

Machine Learning is broadly classified into three categories:

1. Supervised Learning
In supervised learning, the algorithm is trained on a labeled dataset, meaning the input data comes with the correct output. The model learns to map inputs to outputs and can then predict outcomes for new data.
Examples: Predicting house prices